In [1]:
import pandas as pd
from datetime import datetime
from google.colab import drive

import numpy as np

drive.mount('/content/drive')
notebook_directory = "drive/My Drive/Investment/"
data_directory = notebook_directory + "data"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies


def get_data(symbol, data_type, start_date , end_date):
    try:
        import requests 
        import re
        from io import StringIO
        import os
        import os.path
        
        filename = data_directory+"/"+symbol+"_"+data_type
        if not os.path.exists(data_directory):
            os.makedirs(data_directory)
        
        #check cache
        threshold = 10800
        diff = threshold + 1
        if os.path.isfile(filename):
            statbuf = os.stat(filename)
            diff = datetime.now().timestamp() - statbuf.st_mtime
             
        if(diff>(threshold)):
            #crawl
            api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
            (crumb, cookies)=getCrumb(symbol)
            api_url = api_url.format(symbol,0,datetime.now().strftime('%s'),"1d", data_type, crumb)
            r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
            content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
            res = '"'+content+'"'
            
            #save
            text_file = open(filename, "w")
            n = text_file.write(r.content.decode('utf-8'))
            text_file.close()
        
        #load
        df = pd.read_csv(filename)
        df["Date"]=pd.to_datetime(df['Date'])
        df = df[df["Date"]>=start_date][df["Date"]<=end_date]
        return df  
    except:
        print("Error getting data")

def get_price_data(stock, start_date, end_date):
    #print("Getting price data ...")
    price = get_data(stock,"history", start_date, end_date)
    if(price is not None):
        price.rename(columns={'Date':'date','Open':'open','High':'high',
                              'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
        price["date"]=pd.to_datetime(price['date'])
        price = price.set_index(["date"])
        return price
def get_dividend_data(stock, start_date, end_date):
    #print("Getting dividend data ...")
    dividend = get_data(stock,"div", start_date, end_date)
    if(dividend is not None):
        dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
        dividend["date"]=pd.to_datetime(dividend['date'])
        dividend = dividend.set_index(["date"])
        return dividend
def plot_chart(data, height, width):
    print("Ploting chart ...")
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    df = data
    df = df.sort_index(ascending=True)
    fig = go.FigureWidget(make_subplots(rows=1, specs=[[{"secondary_y": True}]]))
    
    
    
    for col in data.columns:
        if("Dividend" in col):
            fig.add_trace(
                go.Scattergl(x=list(df[df[col]>0].index), y=list(df[df[col]>0][col]),
                name = col, marker=dict(color="green", size=2), line = dict(color='grey', width=1, dash='dash')),
                row=1, col=1,
                secondary_y=True
            )
        else:
            fig.add_trace(
                go.Scattergl(x=list(df.index), y=list(df[col]),
                name = col),
                row=1, col=1
            )
            
    
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig['layout'].update(height=height, width=width) 
    return fig

In [3]:
stocks = {
    # Bank
    "D05.SI":"DBS Group Holdings Ltd",
    "O39.SI":"Oversea-Chinese Banking Corporation Limited",
    "U11.SI":"United Overseas Bank Limited",
    
    # REIT
    "XZL.SI":"ARA US Hospitality Trust",
    "A17U.SI":"Ascendas Reit",
    "BMGU.SI":"BHG Retail Reit",
    "K2LU.SI":"Cache Log Trust",
    "J91U.SI":"ESR REIT",
    "C61U.SI":"CapitaCom Trust",
    "C38U.SI":"CapitaMall Trust",
    "CNNU.SI":"Cromwell European",
    "J85.SI":"CDL HTrust",


    "O5RU.SI":"AIMS APAC Reit",
    "LIW.SI":"Eagle US Hospitality Trust",
    "BWCU.SI":"EC World REIT",
    "MXNU.SI":"Elite Commercial REIT",
    "Q5T.SI":"Far East HTrust",
    "AW9U.SI":"First Reit",
    "ND8U.SI":"Frasers Com Tr",
    "J69U.SI":"Frasers Cpt Tr",
    "ACV.SI":"Frasers HTrust",
    "BUOU.SI":"Frasers Log & Ind Tr",
    "UD1U.SI":"IREIT Global",
    "AJBU.SI":"Keppel DC Reit",
    "CMOU.SI":"Keppel Pacific Oak US REIT",
    "K71U.SI":"Keppel Reit",
    "JYEU.SI":"LendLease Global Comm",
    "D5IU.SI":"Lippo Mall Indo Retail Trust",
    "BTOU.SI":"ManulifeReit USD",
    "RW0U.SI":"Mapletree North Asia Comm Tr",
    "ME8U.SI":"Mapletree Ind Tr",
    "M44U.SI":"Mapletree Log Tr",
    "N2IU.SI":"Mapletree Comm Tr",
    "TS0U.SI":"OUE Com Reit",
    "C2PU.SI":"ParkwayLife Reit",
    "OXMU.SI":"Prime US REIT",
    "M1GU.SI":"Sabana Reit",
    "CRPU.SI":"Sasseur REIT",
    "SV3U.SI":"SoilbuildBizReit",
    "SK6U.SI":"SPHREIT",
    "P40U.SI":"StarhillGbl Reit",
    "T82U.SI":"Suntec Reit",
    
    #
    "Z77.SI":"Singtel",
    "BN4.SI":"Keppel Corporation Limited",
    "C31.SI":"CapitaLand Limited",
    "C6L.SI":"Singapore Airlines Limited",
    
    # USA
    "QQQ":"Invesco QQQ Trust",
    "AAPL":"Apple Inc",
    "MSFT":"Microsoft Corporation",
    "TSLA":"Tesla, Inc",
    "GOOGL":"Alphabet Inc",
    "FB":"Facebook",
    "NVDA":"NVIDIA Corporation",
    "AMZN":"Amazon",
    "V":"Visa Inc",
    "INTC" : "Intel",
    "MA":"Mastercard Incorporated",
    "FB":"Facebook",
    "DAL":"Delta Air Lines, Inc",
    "UAL":"United Airlines Holdings Inc",
    "AAL":"American Airlines Group Inc",
    "PYPL":"Paypal Holdings Inc",
    "XOM":"Exxon Mobil Corporation",
    "CVX":"Chevron Corporation",
    "AMD":"Advanced Micro Devices, Inc.",
    "C":"Citigroup Inc.",
    "GS":"Goldman sach",
    "JPM":"JP Morgan",
    "BAC":"Bank of America",
    "MS":"Morgan Stanley",
    "DIS":"Disney",
    "TSN":"Tyson",
    "MU":"Micron Technology",
    "BRK-B":"Berkshire Hathaway Inc. ",
    "AKAM":"Akamai Technologies, Inc",
    "BABA":"Alibaba",
    "ADSK":"Atudodesk",
    "T":"At&T",
    "VZ":"Verizon",
    "WMT": "Walmart",
    "TGT": "Target Corp",
    "CHDN": "churchill downs incorporated",
    "ADBE": "Adobe",
    "TDS": "Telephone Data System",
    "WDC": "Western Digital",
    "ADDYY": "Adidas",
    "NKE":"NIKE",
    "STNE": "StoneCo",
    "OLED":"Universal Display Corporation",
    "HAS":"Hasbro, Inc",
    "FNKO":"FNKO",
    "LRCX":"Lam Research Corporation",
    "RTX":"Raytheon Technologies Corp",
    "HPQ": "hewlett packard inc", 
    "HPE": "hewlett packard enterprise inc", 
    "CAT":"Caterpillar Inc",
    "AXP":"America Express",
    "CRM":"salesforce.com",
    "DD":"dupont de nemours",
    "KO":"Cocacola",
    "HON":"honey well",
    "NOC": "Northrop Grumman Corporation",
    "HD": "HOME DEPOT",

    	
    #mREIT
    "IVR":"Invesco Mortgage Capital Inc",
    "MFA":"MFA Financial, Inc",
    "NYMT":"New York Mortgage Trust, Inc",
    "MITT":"AG Mortgage Investment Trust, Inc",
    "XAN":"Exantas Capital Corp",
    
}
start_date = datetime(2020,2,1)
end_date = datetime(2020,6,1)
dca_date = datetime(2020,3,17)
buy_interval = 1
data = None
for s in stocks:
    stock = s
    price_data = get_price_data(stock, start_date, end_date)
    price_data["ohlc"] = (price_data["open"]+price_data["high"]+price_data["low"]+price_data["close"])/4
    price_data["MA_dip"] = np.nan
    import numpy as np
    index_count = 0
    cost = 0
    buy_count = 0
    for index, row in price_data.iterrows():
      df = price_data.loc[dca_date:end_date]
      if (index in df.index) and (index_count % buy_interval == 0):
        cost = cost + row["ohlc"]
        buy_count = buy_count + 1
        price_data.at[index,'MA_dip']=cost/buy_count 
      index_count = index_count + 1
    result = pd.DataFrame(price_data[-1:][["ohlc","MA_dip"]].values)
    result["code"] = s
    result["stock"] = stocks[s]

    if(data is None):
        data = result
    else:
        None
        data= pd.concat([data, result])

pd.set_option('display.max_rows', data.shape[0]+1)
data=data.rename(columns={0: "ohlc", 1: "MA_dip"})
data=data.set_index(["code"])
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,ohlc,MA_dip,stock
code,,,
D05.SI,19.347501,18.955479,DBS Group Holdings Ltd
O39.SI,8.592500,8.735745,Oversea-Chinese Banking Corporation Limited
U11.SI,19.670000,19.624202,United Overseas Bank Limited
XZL.SI,0.376250,0.380160,ARA US Hospitality Trust
A17U.SI,3.012500,2.807234,Ascendas Reit
BMGU.SI,0.620000,0.558830,BHG Retail Reit
K2LU.SI,0.512500,0.489734,Cache Log Trust
J91U.SI,0.367500,0.323112,ESR REIT
C61U.SI,1.635000,1.516702,CapitaCom Trust


In [4]:
data[data["MA_dip"]>(data["ohlc"]*1)][["ohlc","MA_dip","stock"]]

,ohlc,MA_dip,stock
code,,,
O39.SI,8.592500,8.735745,Oversea-Chinese Banking Corporation Limited
XZL.SI,0.376250,0.380160,ARA US Hospitality Trust
LIW.SI,0.137000,0.142049,Eagle US Hospitality Trust
RW0U.SI,0.805000,0.839468,Mapletree North Asia Comm Tr
TS0U.SI,0.362500,0.365372,OUE Com Reit
C6L.SI,3.630000,5.164384,Singapore Airlines Limited
DAL,22.890000,23.989063,"Delta Air Lines, Inc"
UAL,25.757500,26.934583,United Airlines Holdings Inc
AAL,9.867500,11.154479,American Airlines Group Inc


In [5]:
temp = data
temp["diff"] = ((temp["MA_dip"] - temp["ohlc"])/temp["MA_dip"])*100
temp[temp["diff"]>-1]


,ohlc,MA_dip,stock,diff
code,,,,
O39.SI,8.592500,8.735745,Oversea-Chinese Banking Corporation Limited,1.639754
U11.SI,19.670000,19.624202,United Overseas Bank Limited,-0.233375
XZL.SI,0.376250,0.380160,ARA US Hospitality Trust,1.028404
LIW.SI,0.137000,0.142049,Eagle US Hospitality Trust,3.554348
D5IU.SI,0.129250,0.128048,Lippo Mall Indo Retail Trust,-0.938811
RW0U.SI,0.805000,0.839468,Mapletree North Asia Comm Tr,4.105943
TS0U.SI,0.362500,0.365372,OUE Com Reit,0.786141
P40U.SI,0.457500,0.454920,StarhillGbl Reit,-0.567086
C6L.SI,3.630000,5.164384,Singapore Airlines Limited,29.710883


In [0]:
start_date = datetime(2019,4,1)
end_date = datetime(2020,6,1)
dca_date = datetime(2020,3,19)
buy_interval = 1
stock = "NCBDF"

price_data = get_price_data(stock, start_date, end_date)
price_data["ohlc"] = (price_data["open"]+price_data["high"]+price_data["low"]+price_data["close"])/4
price_data["MA_dip"] = np.nan
index_count = 0
cost = 0
buy_count = 0
pre_index = None
for index, row in price_data.iterrows():
  df = price_data.loc[dca_date:end_date]
  if (index in df.index):
    if (index_count % buy_interval == 0):
      cost = cost + row["ohlc"]
      buy_count = buy_count + 1
      price_data.at[index,'MA_dip']=cost/buy_count 
    else:
      price_data.at[index,'MA_dip']=price_data.at[pre_index,'MA_dip']
  index_count = index_count + 1
  pre_index = index 
price_data

fig = plot_chart(price_data[["MA_dip","ohlc"]], 700, 1300)
from plotly.offline import iplot
import plotly.graph_objs as go
import IPython
from plotly.offline import init_notebook_mode
display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
init_notebook_mode(connected=False)
iplot(fig)


Output hidden; open in https://colab.research.google.com to view.